### **Library Imports**

In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import random as r
import seaborn as sns
import matplotlib.pyplot as plt

from time import time
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import KFold, GridSearchCV

from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, classification_report

### **Configuration**

In [2]:
class CFG(object):
    def __init__(self,
                 seed: int = 42,
                 n_splits: int = 5,
                 show_info: bool = False,
                 ):

        self.seed = seed
        self.n_splits = n_splits
        self.show_info = show_info
        self.train_data_read_path = "../input/tab-hack-20/Week8_train.csv"
        self.test_data_read_path = "../input/tab-hack-20/Week8_test.csv"
        self.model_save_path = "models"
        if not os.path.exists(self.model_save_path): os.makedirs(self.model_save_path)

cfg = CFG(seed=42, show_info=True)

### **Helpers**

In [3]:
def breaker(num: int=50, char: str="*") -> None:
    print("\n" + num*char + "\n")

    
def get_object_columns(df) -> list:
    object_columns: list = []
    for col in df.columns:
        if df[col].dtype == "object":
            object_columns.append(col)

    return object_columns
    

def print_scores(accuracy: float, auc: float, precision: np.ndarray, recall: np.ndarray, f_score: np.ndarray) -> None:
    print(f"Accuracy  : {accuracy:.5f}")
    print(f"ROC-AUC   : {auc:.5f}")
    print(f"Precision : {precision}")
    print(f"Recall    : {recall}")
    print(f"F-Score   : {f_score}")
    

def get_scores(y_true: np.ndarray, y_pred: np.ndarray) -> tuple:
    accuracy = accuracy_score(y_pred, y_true)
    auc = roc_auc_score(y_pred, y_true)
    precision, recall, f_score, _ = precision_recall_fscore_support(y_pred, y_true)

    return accuracy, auc, precision, recall, f_score

### **Model**

In [4]:
class Model(object):
    def __init__(self, model_name: str, preprocessor, seed: int):
        self.model_name = model_name

        if self.model_name == "lgr":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", LogisticRegression(random_state=seed)),
                ]
            )
        
        elif self.model_name == "knc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", KNeighborsClassifier()),
                ]
            )

        
        elif self.model_name == "dtc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", DecisionTreeClassifier(random_state=seed)),
                ]
            )

        elif self.model_name == "etc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", ExtraTreeClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "rfc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", RandomForestClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "gbc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", GradientBoostingClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "abc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", AdaBoostClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "etcs":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", ExtraTreesClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "gnb":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", GaussianNB()),
                ]
            )
        
        elif self.model_name == "xgbc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", XGBClassifier(random_state=seed)),
                ]
            )

### **Train**

In [5]:
df = pd.read_csv(cfg.train_data_read_path)
df = df.drop(columns=["id", "PARTY_ID"])
object_columns = get_object_columns(df)
df = df.drop(columns=object_columns)

if cfg.show_info:
    breaker()
    for val in set(df.TARGET):
        print(f"Class {val} count : {df[df.TARGET == val].shape[0]}")

X = df.iloc[:, :-1].copy().values
y = df.iloc[:, -1].copy().values

features = [i for i in range(X.shape[1])]


**************************************************

Class 0 count : 89367
Class 1 count : 21532


In [6]:
names: list = ["lgr", "knc", "gnb", "dtc", "etc", "abc", "gbc", "etcs", "rfc", "xgbc"]
# names: list = ["lgr"]

feature_transformer = Pipeline(
    steps=[
        ("Simple_Imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
        ("Standard_Scaler", StandardScaler())
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("features", feature_transformer, features),
    ]
)

best_auc = 0.0
for name in names:
    fold = 1
    if cfg.show_info: breaker()
    for tr_idx, va_idx in KFold(n_splits=cfg.n_splits, random_state=cfg.seed, shuffle=True).split(X):
        X_train, X_valid, y_train, y_valid = X[tr_idx], X[va_idx], y[tr_idx], y[va_idx]
        my_pipeline = Model(name, preprocessor, cfg.seed)
        my_pipeline.model.fit(X_train, y_train)

        y_pred = my_pipeline.model.predict(X_valid)
        if cfg.show_info:
            acc, auc, pre, rec, f1 = get_scores(y_valid, y_pred)
            print(f"{my_pipeline.model_name}, {fold}\n")
            print_scores(acc, auc, pre, rec, f1)
            print("")
        else:
            _, auc, _, _, _ = get_scores(y_valid, y_pred)

        if auc > best_auc:
            best_auc = auc
            model_fold_name = f"{name}_{fold}"
            
            with open(os.path.join(cfg.model_save_path, f"best_model.pkl"), "wb") as fp:
                pickle.dump(my_pipeline.model, fp)
        fold += 1
    

if cfg.show_info: 
    breaker()
    print(f"Best Model : {model_fold_name.split('_')[0]}, Best Fold : {model_fold_name.split('_')[1]}")

breaker()


**************************************************

lgr, 1

Accuracy  : 0.83142
ROC-AUC   : 0.81392
Precision : [0.99007195 0.16231475]
Recall    : [0.83291104 0.79493088]
F-Score   : [0.90471701 0.2695839 ]

lgr, 2

Accuracy  : 0.83318
ROC-AUC   : 0.81651
Precision : [0.99014257 0.16595644]
Recall    : [0.83461647 0.79840547]
F-Score   : [0.90575169 0.2747942 ]

lgr, 3

Accuracy  : 0.83039
ROC-AUC   : 0.82790
Precision : [0.99177438 0.16105825]
Recall    : [0.83059187 0.82520809]
F-Score   : [0.90405509 0.26951456]

lgr, 4

Accuracy  : 0.82534
ROC-AUC   : 0.81431
Precision : [0.99061534 0.15461802]
Recall    : [0.82624795 0.80236686]
F-Score   : [0.90099668 0.25927342]

lgr, 5

Accuracy  : 0.82691
ROC-AUC   : 0.82042
Precision : [0.99124383 0.15585606]
Recall    : [0.82743757 0.81339713]
F-Score   : [0.90196379 0.26158877]


**************************************************

knc, 1

Accuracy  : 0.80564
ROC-AUC   : 0.65201
Precision : [0.96123599 0.14937662]
Recall    : [0.82657074 0

### **Predict**

In [7]:
# df = pd.read_csv(cfg.test_data_read_path)

# # Drop all categorical features
# df = df.drop(columns=["id", "PARTY_ID"])

# X = df.copy().values

# model = pickle.load(open(os.path.join(cfg.model_save_path, f"best_model.pkl"), "rb"))
# y_pred = model.predict_proba(X)[:, 1]

# ss_df = pd.DataFrame({
#     "Id" : pd.RangeIndex(1,10001),
#     "Category" : y_pred.reshape(-1).astype("uint8")
# })
# ss_df["TARGET"] = y_pred
# ss_df.to_csv("submission.csv", index=False)